In [5]:
import sys, os, time
sys.path.append('../..')
import pyzx as zx
from benchmarking import benchmark
sys.path.append('/Users/calum/Developer/pyzx-heuristics-master')
import pyzx_heur as zx_heuristics

In [6]:
b = benchmark(dirpath='benchmark')
# b = benchmark()
# b.load_circuits(os.path.join('..', '..', 'circuits', 'benchmarking', 'Fast', 'before'), group_name='fast')
# b.load_circuits(os.path.join('..', '..', 'circuits', 'benchmarking', 'Fast', 'nrscm'), group_name='fast', simp_strategy='NRSCM')
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_opt']
Loaded functions:  ['zx_heur', 'zx_heur_nu', 'F_c_00_ES', 'F_c_11_ES', 'F_c_22_ES', 'F_c_33_ES', 'F_c_44_ES', 'F_c_55_ES', 'F_g_00_EC', 'F_g_11_EC', 'F_c_66_ES', 'F_g0_c0', 'F_g0_c1', 'F_g0_c2', 'F_g0_c3']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,F_c_00_ES,F_c_11_ES,F_c_22_ES,F_c_33_ES,F_c_44_ES,F_c_55_ES,F_c_66_ES,F_g0_c0,F_g0_c1,F_g0_c2,F_g0_c3,F_g_00_EC,F_g_11_EC,NRSCM,zx_heur,zx_heur_nu
fast,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y


In [7]:
def basic_optimize(c):
    c1 = zx.optimize.basic_optimization(c.copy(), do_swaps=False).to_basic_gates()
    c2 = zx.optimize.basic_optimization(c.copy(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2
    return c1

def full_reduce(c):
    g = c.to_graph()
    zx.simplify.full_reduce(g,quiet=True)
    c2 = zx.extract_circuit(g,up_to_perm=False).to_basic_gates()
    return basic_optimize(c2)

def flow_optimize(c):
    g = c.to_graph()
    zx.simplify.teleport_reduce(g, store=False)
    zx.simplify.to_graph_like(g, assert_bound_connections=False)
    zx.simplify.flow_2Q_simp(g, cFlow=False, rewrites=['id_fuse','lcomp','pivot'], max_lc_unfusions=0, max_p_unfusions=0)
    g.place_tracked_phases()
    # c2 = zx.extract_simple(g,up_to_perm=False).to_basic_gates()
    c2 = zx.extract_circuit(g,up_to_perm=False).to_basic_gates()
    
    g2 = c2.to_graph()
    zx.simplify.teleport_reduce(g2, store=False)
    zx.simplify.to_graph_like(g2, assert_bound_connections=False)
    zx.simplify.flow_2Q_simp(g2, cFlow=True, rewrites=['id_fuse','lcomp','pivot'], max_lc_unfusions=5, max_p_unfusions=5)
    g2.place_tracked_phases()
    c3 = zx.extract_simple(g2,up_to_perm=False).to_basic_gates()
    # c2 = zx.extract_circuit(g,up_to_perm=False).to_basic_gates()
    return basic_optimize(c3)

In [87]:
b.add_simplification_func(flow_optimize, 'F_g0_c5', groups_to_run=['fast'], verify=True)

Processing F_g0_c5 on ham15-low                                       :  84%|████████▍ | 26/31 [1:15:23<31:58, 383.64s/it]   

In [4]:
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_opt']
Loaded functions:  ['zx_heur', 'zx_heur_nu', 'F_c_00_ES', 'F_c_11_ES', 'F_c_22_ES', 'F_c_33_ES', 'F_c_44_ES', 'F_c_55_ES', 'F_g_00_EC', 'F_g_11_EC', 'F_c_66_ES', 'F_g0_c0', 'F_g0_c1', 'F_g0_c2', 'F_g0_c3']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,F_c_00_ES,F_c_11_ES,F_c_22_ES,F_c_33_ES,F_c_44_ES,F_c_55_ES,F_c_66_ES,F_g0_c0,F_g0_c1,F_g0_c2,F_g0_c3,F_g_00_EC,F_g_11_EC,NRSCM,zx_heur,zx_heur_nu
fast,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y


In [3]:
df = b.df(groups=['fast'],
     routines=['all'],
     funcs=['all'],
     atts=['Qubits','2Q Count'])

In [84]:
df = df.drop(['Adder8','QFT8','QFTAdd8','grover_5','ham15-low','hwb6','qft_4','tof_3','barenco_tof_3'])

In [85]:
for strategy in df.columns.levels[0]:
    if strategy not in ["Original","Circuits"]:
        percentage_reduction = ((df["Original", "2Q Count"] - df[strategy, '2Q Count']) / df["Original", "2Q Count"]) * 100
        print(f"{strategy}: {percentage_reduction.mean():.2f}")


F_LC5: 13.45
F_c_00_ES: 14.70
F_c_11_ES: 16.76
F_c_22_ES: 19.35
F_c_33_ES: 19.52
F_c_44_ES: 19.54
F_c_55_ES: 19.55
F_g0_c0: 15.48
F_g0_c1: 16.99
F_g0_c2: 19.03
F_g0_c3: 19.45
F_g0_c4: 19.27
F_g_00_EC: 15.43
NRSCM: 18.47
T_LC5: 13.36
zx_heur: 11.02
zx_heur_nu: 14.57


In [24]:
b.save('benchmark')